<a href="https://colab.research.google.com/github/ailuj/BayesianDeepLearning/blob/playground/MCDropout_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive',force_remount=True)


Mounted at /content/drive


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"batuhanipekci","key":"8d50f8f4e095ae646c2912649292ed60"}'}

In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
from google.colab import files
files.upload()

In [3]:
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d wendykan/lending-club-loan-data 


 98% 236M/240M [00:02<00:00, 112MB/s] 
100% 240M/240M [00:02<00:00, 102MB/s]


In [4]:
from zipfile import ZipFile
zip_file = ZipFile('lending-club-loan-data.zip')
zip_file.namelist()
loan = pd.read_csv(zip_file.open('loan.csv'))

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [32]:
loan = loan[(loan['loan_status'] != 'Current')]
selected = loan[['total_pymnt', 'annual_inc', 'int_rate','loan_amnt','purpose']]
selected = selected.dropna()
features = selected[["annual_inc", "int_rate", "purpose"]]
features = pd.get_dummies(features)
target = selected.apply(lambda x: ((x['total_pymnt'] - x['loan_amnt'])/x['loan_amnt']), axis = 1)

features.head()

,annual_inc,int_rate,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,24000.0,10.65,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,30000.0,15.27,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,12252.0,15.96,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,49200.0,13.49,0,0,0,0,0,0,0,0,0,1,0,0,0,0
5,36000.0,7.90,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [12]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=42)

scaler_tr = preprocessing.MinMaxScaler().fit(X_train)
X_train = scaler_tr.transform(X_train)

scaler_test = preprocessing.MinMaxScaler().fit(X_test)
X_test = scaler_test.transform(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


# MLP Architecture


In [13]:
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers.core import Dense, Activation, Dropout
import time

Using TensorFlow backend.


MC Dropout Settings

In [0]:
dropout_prob = 0.20 # dropout should be between 0.05 and 0.5
n_epochs = 100
tau = 1.0

N = X_train.shape[0]
batch_size = 500
lengthscale = 1e-2

# Define the weight-decay regularizer
reg = lengthscale**2 * (1 - dropout_prob) / (2. * N * tau)
        
n_hidden = [100,100] # 2 hidden layers, each with 100 nodes
N = X_train.shape[0]
input_dim = X_train.shape[1]
output_dim = 1



In [0]:
from keras import Model, Input
from keras.layers import Dropout
from keras.layers import Dense
from keras.regularizers import l2

def mlp(input_dim, n_hidden, dropout_prob, train_activation, MCDropout = False):
  inputs = Input(shape=(X_train.shape[1],))
  inter = Dropout(dropout_prob)(inputs, training=True)
  inter = Dense(n_hidden[0], activation=train_activation,
                W_regularizer=l2(reg))(inter)
  for i in range(len(n_hidden) - 1):
    inter = Dropout(dropout_prob)(inter, training=True)
    inter = Dense(n_hidden[i+1], activation=train_activation,
                  W_regularizer=l2(reg))(inter)
  if MCDropout == True:
    inter = Dropout(dropout_prob)(inter, training=True)
  outputs = Dense(output_dim,W_regularizer=l2(reg))(inter) 
  model = Model(inputs, outputs)
  
  return model

In [19]:
from keras.layers.advanced_activations import LeakyReLU

model = mlp(input_dim = input_dim, n_hidden = n_hidden, 
            dropout_prob = dropout_prob, train_activation = 'relu',MCDropout = False)
model.compile(optimizer='sgd', loss='mse', metrics=['mae'])

model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=n_epochs, verbose=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(100, activation="relu", kernel_regularizer=<keras.reg...)`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(100, activation="relu", kernel_regularizer=<keras.reg...)`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/100
191349/191349 [==============================] - 3s 18us/step - loss: 0.1418 - mean_absolute_error: 0.2975
Epoch 2/100
191349/191349 [==============================] - 2s 9us/step - loss: 0.1393 - mean_absolute_error: 0.2955
Epoch 3/100
191349/191349 [==============================] - 2s 9us/step - loss: 0.1387 - mean_absolute_error: 0.2949
Epoch 4/100
191349/191349 [==============================] - 2s 9us/step - loss: 0.1385 - mean_absolute_error: 0.2944
Epoch 5/100
191349/191349 [==============================] - 2s 9us/step - loss: 0.1380 - mean_absolute_error: 0.2943
Epoch 6/100
191349/191349 [==============================] - 2s 8us/step - loss: 0.1379 - mean_absolute_error: 0.2939
Epoch 7/100
191349/191349 [==============================] - 2s 9us/step - loss: 0.1378 - mean_absolute_error: 0.2939
Epoch 8/100
191349/191349 [==============================] - 2s 9us/step - loss: 0.1377 - mean_absolute_error: 0.2938
Epoch 9/100
191349/191349 [============================

In [20]:
pred = model.predict(X_test, verbose=1)

94247/94247 [==============================] - 3s 27us/step


In [21]:
from sklearn.metrics import mean_squared_error
import numpy as np
mean_squared_error(pred, y_test)

0.1364830229076091

In [26]:
list(zip(y_test, pred))

[(0.10550598290598291, array([-0.0485553], dtype=float32)),
 (-0.770346875, array([-0.06272724], dtype=float32)),
 (0.11473000000000001, array([-0.04236054], dtype=float32)),
 (0.1290037725175037, array([-0.07933842], dtype=float32)),
 (0.013004999999999928, array([-0.04852194], dtype=float32)),
 (0.10416958321666668, array([-0.06495208], dtype=float32)),
 (0.15396333333333329, array([-0.02237507], dtype=float32)),
 (-0.823771, array([-0.09241371], dtype=float32)),
 (0.08348354430379752, array([-0.09133898], dtype=float32)),
 (-0.701518158567775, array([-0.04463474], dtype=float32)),
 (-0.8016984881209503, array([-0.06482902], dtype=float32)),
 (0.06073666666666662, array([-0.06243499], dtype=float32)),
 (-0.28588199999999997, array([-0.11443646], dtype=float32)),
 (0.08230638297872332, array([-0.07760439], dtype=float32)),
 (0.08284244604316549, array([-0.12172892], dtype=float32)),
 (0.15752666666666676, array([-0.06050261], dtype=float32)),
 (0.40044828427142865, array([-0.07031765]

# Bayesian MLP with Dropout Trick

In [28]:
model = mlp(input_dim = input_dim, n_hidden = n_hidden, 
            dropout_prob = dropout_prob, train_activation = 'relu', 
            MCDropout = True)
model.compile(optimizer='sgd', loss='mse', metrics=['mae'])

model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=n_epochs, verbose=0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(100, activation="relu", kernel_regularizer=<keras.reg...)`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(100, activation="relu", kernel_regularizer=<keras.reg...)`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


In [29]:
T = 1000
        
probs = []
for _ in range(T):
    probs += [model.predict(X_test,verbose=1)]

94247/94247 [==============================] - 3s 34us/step


In [30]:
import numpy as np
predictive_mean = np.mean(probs, axis=0)
predictive_variance = np.var(probs, axis=0)
                                  
l = 100 
# I need to check out how to choose l and tau

#tau = l**2 * (1 - dropout_prob) / (2 * N * 1)
#predictive_variance += tau**-1




0.1354552632402013

In [31]:
# Accuracy
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,predictive_mean)


0.1360733444152823